In [ ]:
%load_ext autoreload
%autoreload 2

# Plotting some traces

In [ ]:
import data
import pylab as plt
import torch
import seaborn as sns
import pandas as pd
import numpy as np

sensors = ['ALNL', 'ALNR', 'ASKL', 'ASKR', 'BAGL', 'OLQDL', 'OLQDR', 'OLQVL', 'OLQVR', 'PHAR']

plt.figure(figsize=(30,8))
traces = data.get_data()#[sensors + ['AVAR']]
traces.index = np.arange(len(traces))
dvdt = data.get_deriv()
print(traces.shape, dvdt.shape)

avaup = traces['AVAL']>0.6
avarising = dvdt['AVAR'] > 0.02
avarise = avarising.astype(int).diff() > 0
weights = avaup*2 + 1

# sns.heatmap(traces.corr(), cmap='bwr', vmin=-1, vmax=1, annot=True)
# plt.plot(traces['ALNR'].values)
# plt.plot((traces['OLQDR'].values - traces['ALNR'].values) * 5)
# plt.plot((traces['BAGL'].values) * 5)
# plt.plot(traces['ASKR'].values )
# for i in range(0,3500,150):
#     plt.axvline(i, color='k')
# plt.plot((traces['SMDDR'].values) * 2)
# plt.plot((traces['OLQVR'].values) * 3)
plt.plot(traces['VA01'], 'r')
plt.plot(traces['AVEL'])
plt.plot(traces['AVER'])
plt.plot(traces['AVAL'])
plt.plot(avarising)
plt.plot( avarise )
# plt.plot(dvdt['AVAL']>0.02)
# plt.plot(traces['PHAR']*3)

# plt.plot(weights)

cor = traces[sensors + ['AVAL']].corr()
# sns.heatmap(cor, cmap='seismic', vmin=-1, vmax=1, annot=True)

plt.legend()
plt.show()


risewhy = pd.DataFrame(0., index=np.arange(20), columns=traces.columns)
for i in np.argwhere(avarise.values > 0.5):
    i = i[0]
    risewhy = pd.DataFrame(risewhy.values + traces.iloc[i-10:i+10, :])
risewhy /= avarise.sum()
    
# plt.figure(figsize=(3,25))
# for i,c in enumerate(risewhy.columns):
#     plt.plot(risewhy[c] + 0.5*i)
# plt.axvline(3203, color='k', linestyle='--')
# plt.show()

# plt.plot(traces['VA01'])
# plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import KFold
import pandas as pd
from copy import copy
import itertools
import numpy as np

def list_of_combs(arr):
    """returns a list of all subsets of a list"""
    
    combs = []
    for i in range(0, len(arr)+1):
        listing = [list(x) for x in itertools.combinations(arr, i)]
        combs.extend(listing)
    return combs

def predict(pred=sensors, x=None, yi='AVAR', y=None, w=None, deriv=False):

#     for preds in list_of_combs(pred)[1:]:
    
    preds = pred
    df = data.get_data()
    df.index = np.arange(len(df))
    X = df[preds]
    names = preds
    if deriv:
        X = pd.concat([X, dvdt[preds]], axis=1)
        X.fillna(0, inplace=True)
        names = names + ['d%s/dt'%n for n in preds]
    if y is None:
        y = df[yi]


    model = LinearRegression(normalize = True)
#     model = LogisticRegression(tol = 1e-7)
    model.fit(X, y, w)
#     print(model.coef_.shape, len(names))
    print(pd.DataFrame(model.coef_[None], columns=names))
    score = model.score(X, y)

    plt.figure(figsize=(15,5))
    plt.plot(np.arange(len(y)), y, 'r')
#   out = model.predict_proba(X)[:,1]
    out = model.predict(X)
    plt.plot(out, 'k')
#     plt.axhline(0.5, color='k', linestyle='--')
#     plt.plot(model.predict(X))
    print(model.predict(X).sum())
    plt.show()
    return probs
    
sens = sensors.copy()
sens.remove('OLQDL')
sens.remove('OLQVL')
sens.remove('OLQDR')
# sens.remove('OLQVR')
# predict(pred=sens, yi='AVAR')
p = predict(pred=['AVAL', 'AVBL', 'AVEL'], yi='ALNL', deriv=True)

In [ ]:
from sklearn.metrics import roc_curve

roc = roc_curve(avaup, p, sample_weight=weights)
plt.plot(roc[0], roc[1])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')

In [ ]:

# LSTM for predicting AVA up from sensory neurons traces
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, log_loss
from keras.utils import to_categorical
# convert an array of values into a dataset matrix

idx_tr = []
idx_te = []

class Test(Callback):

    def on_epoch_end(self, epoch, logs=None):
        if epoch%10 != 0:
            return
        # make predictions
        trainPredict = model.predict(trainX)
        testPredict = model.predict(testX)

        # plot baseline and predictions
        plt.figure(figsize=(12,3))
        plt.plot(y[:,-1], 'g')
        plt.plot(idx_tr, trainPredict[:,-1], 'ko', ms=1)
        plt.plot(idx_te, testPredict[:,-1], 'ro', ms=1)
        plt.show()
        print('test loss: ', log_loss(testY, testPredict))

def create_dataset(X, y, look_back=10, offset=0):
    dataX, dataY, idx = [], [], []
    for i in range(len(y)-look_back-1):
        # sensors from prev timesteps
#         if avaup[offset+i+look_back-1]:
#             continue
        a = X[i:(i+look_back)]
        dataX.append(a)
        # AVA up at t+1
        dataY.append(y[i + look_back])
        idx.append(offset + i + look_back)
    return numpy.array(dataX), numpy.array(dataY), idx
# fix random seed for reproducibility
numpy.random.seed(7)
# normalize the dataset
X = data.get_data()[['ASKL', 'ASKR']].values
# scaler = MinMaxScaler(feature_range=(0, 1))
# X = scaler.fit_transform(X)
print(X.shape)

# y = data.get_data()[['AVAR']].values

y = avarise.values
y = to_categorical(y)
print(y.shape)

# split into train and test sets
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size
xtr, ytr, xte, yte = X[-train_size:,:], y[-train_size:], X[:-train_size,:], y[:-train_size]
# reshape into X=t and Y=t+1
look_back = 150
trainX, trainY, idx_tr = create_dataset(xtr, ytr, look_back, offset = test_size)
testX, testY, idx_te = create_dataset(xte, yte, look_back)
# reshape input to be [samples, time steps, features] already done
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(10, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(LSTM(5))
model.add(Dense(y.shape[-1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
test = Test()
callbacks_list = [test]
# fit the model
print(trainX.shape)
model.fit(trainX, trainY, epochs=1000, batch_size=64, callbacks=callbacks_list, verbose=2)


In [ ]:
from numpy.random import rand

print(trainX.shape, testY.shape)
X = testX[:look_back,0,:]
print(X.shape)
res = X

n_pred = 1300
for i in range(n_pred):
    X = np.array(res[-look_back:])[None]
    x = model.predict(X)
    nxt = testX[look_back+i,0,:]
    nxt[-1] = x
    res = np.concatenate((res, nxt[None]))
print(res.shape)
plt.plot(res[:,-1], 'r')
plt.plot(np.arange(look_back, look_back+n_pred), testY[:n_pred], color='k', linestyle='--')
plt.show()

In [ ]:
"""Autoencoder"""

# LSTM for predicting AVA up from sensory neurons traces
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, log_loss
from keras.utils import to_categorical
from keras.callbacks import Callback
# convert an array of values into a dataset matrix

idx_tr = []
idx_te = []

class Test(Callback):

    def on_epoch_end(self, epoch, logs=None):
        if epoch%10 != 0:
            return
        # make predictions
        trainPredict = model.predict(trainX)
        testPredict = model.predict(testX)

        # plot baseline and predictions
        plt.figure(figsize=(12,3))
        plt.plot(X[:,-1], 'g')
        plt.plot(trainPredict[:,-1], 'ko', ms=1)
        plt.plot(testPredict[:,-1], 'ro', ms=1)
        plt.show()
        print('test loss: ', log_loss(testY, testPredict))

def create_dataset(X, look_back=10):
    dataX = []
    for i in range(len(y)-look_back-1):
        a = X[i:(i+look_back)]
        dataX.append(a)
        
    return numpy.array(dataX)
# fix random seed for reproducibility
numpy.random.seed(7)
# normalize the dataset
X = data.get_data().values
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
print(X.shape)

# y = data.get_data()[['AVAR']].values

y = X 
print(y.shape)

# split into train and test sets
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size
xtr, xte = X[-train_size:,:], X[:-train_size,:]
# reshape into X=t and Y=t+1
look_back = 150
trainX = create_dataset(xtr, look_back)
testX = create_dataset(xte, look_back)
# reshape input to be [samples, time steps, features] already done
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(10, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(LSTM(5))
model.add(LSTM(10))
model.add(LSTM(trainX.shape[1], activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
test = Test()
callbacks_list = [test]
# fit the model
print(trainX.shape)
model.fit(trainX, trainX, epochs=1000, batch_size=64, callbacks=callbacks_list, verbose=2)
